In [ ]:
pip install transformers datasets torch tqdm peft bitsandbytes accelerate autoawq compressed-tensors openpyxl evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# задаем диапазон строк для дообчения
start = 4500
end = 5000

In [ ]:
import openpyxl

def read_xlsx_to_nested_list(file_path):
    all_lists = []
    current_list = []
    headers_length = 0

    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    for row in sheet.iter_rows(values_only=True):
        if all(cell is None for cell in row):
            if current_list:
                all_lists.append(current_list)
                current_list = []
            continue

        if row[0] == "Полное название":
            current_list.append([cell for cell in row if cell is not None])
            headers_length = len(current_list[0])
        else:
            if current_list:
                current_row = []
                for i in range(headers_length):
                    if i < len(row) and row[i] is not None:
                        current_row.append(row[i])
                    else:
                        current_row.append('')
                current_list.append(current_row)

    if current_list:
        all_lists.append(current_list)

    all_lists = [lst for lst in all_lists if lst]

    return all_lists

file_path = '/content/drive/MyDrive/dataset_generated.xlsx'
result = read_xlsx_to_nested_list(file_path)

def format_data_to_dict(nested_list):
    result = []

    for sublist in nested_list[1:]:
        question_text = sublist[0]

        answer_parts = []
        for i in range(1, len(sublist)):
            header = nested_list[0][i]
            value = sublist[i]
            answer_parts.append(f"{header}:{value}")

        answer_text = "/sprt/".join(answer_parts)

        result.append({
            "input": question_text,
            "output": answer_text
        })

    return result

resulted = []
for array in result:
    a = format_data_to_dict(array)
    for larray in a:
        resulted.append(larray)

res_len = int(len(resulted))
data1 = resulted[start:res_len if res_len < end else end:3]
data2 = resulted[start + 1:res_len if res_len < end else end:3]
data = data1 + data2
validation_data = resulted[start + 2:res_len if res_len < end else end:3]

print(data)
print(validation_data)

[{'input': 'Резец расточной, ГОСТ 18878-73', 'output': 'Категория:Резец/sprt/Тип:расточной/sprt/Форма:/sprt/Материал:/sprt/Размер:/sprt/Стандарт:ГОСТ 18878-73/sprt/Индекс заполненности:0'}, {'input': 'Резец токарный, ГОСТ 18877-73, изогнутый', 'output': 'Категория:Резец/sprt/Тип:токарный/sprt/Форма:изогнутый/sprt/Материал:/sprt/Размер:/sprt/Стандарт:ГОСТ 18877-73/sprt/Индекс заполненности:0'}, {'input': 'Резец фрезерный, 10 мм', 'output': 'Категория:Резец/sprt/Тип:фрезерный/sprt/Форма:/sprt/Материал:/sprt/Размер:10 мм/sprt/Стандарт:/sprt/Индекс заполненности:0'}, {'input': 'Резец сталь, ГОСТ 18878-73, канавочный, 20 мм', 'output': 'Категория:Резец/sprt/Тип:канавочный/sprt/Форма:/sprt/Материал:сталь/sprt/Размер:20 мм/sprt/Стандарт:ГОСТ 18878-73/sprt/Индекс заполненности:0'}, {'input': 'Резец прямой, сталь, 10 мм, фрезерный', 'output': 'Категория:Резец/sprt/Тип:фрезерный/sprt/Форма:прямой/sprt/Материал:сталь/sprt/Размер:10 мм/sprt/Стандарт:/sprt/Индекс заполненности:0'}, {'input': 'Резец

In [ ]:
# настраиваем параметры
training_args = TrainingArguments(
    output_dir="./drive/MyDrive/pp_4sem/results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=15,
    learning_rate=5e-4,
    lr_scheduler_type="cosine",
    fp16=True,
    logging_dir="./logs",
    logging_strategy="epoch",
    report_to="tensorboard",
    save_total_limit=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="rouge-L",
    greater_is_better=True,
)

lora_config = LoraConfig(
    r=2,
    lora_alpha=4,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import numpy as np
import evaluate
import math
import os

MODEL_NAME = "TheBloke/Llama-2-7B-Chat-AWQ"
MODEL_SAVE_PATH = "./drive/MyDrive/pp_4sem/fine_tuned_model"

dataset = Dataset.from_list(data)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    inputs = [
        f"Вопрос: {inp}\nОтвет: {out}"
        for inp, out in zip(examples["input"], examples["output"])
    ]
    tokens = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True)

validation_dataset = Dataset.from_list(validation_data)
tokenized_validation_datasets = validation_dataset.map(tokenize_function, batched=True)

if (start == 0 or not os.path.exists(MODEL_SAVE_PATH)):
  print("Загружаем исходную модель...")
  model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="cuda:0"
  )
else:
  print("Загружаем дообученную модель...")
  model = AutoModelForCausalLM.from_pretrained(
        MODEL_SAVE_PATH,
        torch_dtype=torch.float16,
        device_map="cuda:0"
  )

model = get_peft_model(model, lora_config)

# Метрики
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

# Добавляем кастомные метрики
def extract_key_value_pairs(text):
    """Парсит строку вида 'ключ1:значение1/sprt/ключ2:значение2' в словарь"""
    pairs = text.split('/sprt/')
    result = {}
    for pair in pairs:
        if ':' in pair:
            key, value = pair.split(':', 1)
            result[key.strip()] = value.strip()
    return result

def calculate_metrics(pred_dict, true_dict):
    """Вычисляет TP, FP, FN для пар ключ-значение"""
    tp = 0  # Полностью совпадающие пары
    fp = 0  # Лишние пары в предсказании
    fn = 0  # Пропущенные пары из истинных данных

    for key, true_value in true_dict.items():
        pred_value = pred_dict.get(key)
        if pred_value == true_value:
            tp += 1
        else:
            fn += 1

    for key in pred_dict:
        if key not in true_dict:
            fp += 1

    return tp, fp, fn

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Перплексия
    logits_tensor = torch.tensor(logits)
    labels_tensor = torch.tensor(labels)
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(logits_tensor.view(-1, logits_tensor.size(-1)), labels_tensor.view(-1))
    perplexity = math.exp(loss.item())

    # Декодируем тексты
    pred_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Вычисляем F1 и Key-Value Accuracy
    total_tp = total_fp = total_fn = 0
    kv_accuracy_count = 0

    for pred_text, true_text in zip(pred_texts, label_texts):
        pred_dict = extract_key_value_pairs(pred_text)
        true_dict = extract_key_value_pairs(true_text)

        # Считаем полностью совпадающие пары
        if pred_dict == true_dict:
            kv_accuracy_count += 1

        # Считаем TP/FP/FN
        tp, fp, fn = calculate_metrics(pred_dict, true_dict)
        total_tp += tp
        total_fp += fp
        total_fn += fn

    # Рассчитываем Precision, Recall, F1
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp +
    total_fn) if (total_tp + total_fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Key-Value Accuracy
    kv_accuracy = kv_accuracy_count / len(pred_texts) if len(pred_texts) > 0 else 0

    # Существующие метрики
    rouge_scores = rouge_metric.compute(predictions=pred_texts, references=label_texts)
    meteor_score = meteor_metric.compute(predictions=pred_texts, references=label_texts)

    return {
        "perplexity": perplexity,
        "rouge-L": rouge_scores["rougeL"],
        "meteor": meteor_score["meteor"],
        "kv_precision": precision,
        "kv_recall": recall,
        "kv_f1": f1,
        "kv_accuracy": kv_accuracy
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_validation_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
trainer.save_model(MODEL_SAVE_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/262 [00:00<?, ? examples/s]

Map:   0%|          | 0/131 [00:00<?, ? examples/s]

Загружаем дообученную модель...


/usr/local/lib/python3.11/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` b

Epoch,Training Loss,Validation Loss,Perplexity,Rouge-l,Meteor,Kv Precision,Kv Recall,Kv F1,Kv Accuracy
1,1.140200,0.366535,4804731.928794,0.774404,0.664906,0.698347,0.608643,0.650417,0.000000
2,0.231000,0.151820,14330128.340843,0.851137,0.795748,0.992470,0.791116,0.880428,0.000000
3,0.122100,0.135043,40558007.434530,0.846016,0.802337,0.992690,0.815126,0.895188,0.000000
4,0.106300,0.127296,57351298.297633,0.845898,0.792804,0.992636,0.809124,0.891534,0.000000


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import evaluate

MODEL_PATH = "./drive/MyDrive/pp_4sem/fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH).to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

def generate_with_few_shot(model, tokenizer, input_text, examples, max_new_tokens=256):
    few_shot_prompt = ""
    for ex in examples:
        few_shot_prompt += f"INPUT: {ex['input']}\nOUTPUT: {ex['output']}{tokenizer.eos_token}\n\n"

    few_shot_prompt += f"INPUT: {input_text}\nOUTPUT:"

    inputs = tokenizer(few_shot_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Удаляем few-shot префикс
    result = decoded[len(few_shot_prompt):].strip()

    # Обрезаем всё, что после следующего 'INPUT:' (если вдруг модель продолжила)
    result = result.split("INPUT:")[0].strip()

    return result

examples = [
    {'input': 'Резец расточной, ГОСТ 18878-73', 'output': 'Категория:Резец/sprt/Тип:расточной/sprt/Форма:/sprt/Материал:/sprt/Размер:/sprt/Стандарт:ГОСТ 18878-73/sprt/Индекс заполненности:0'},
]

test_input = "Резец прямой, круглый, стальной, ГОСТ 188131231238-73"

result = generate_with_few_shot(model, tokenizer, test_input, examples)
print("Generated:\n", result)

Generated:
 TPUT: Категория:Резец/sprt/Тип:прямой/sprt/Форма:круглый/sprt/Материал:стальный/sprt/Размер:/sprt/Стандарт:ГОСТ 188131231238-73/sprt/Индекс заполненности:0</s>
